In [68]:
import numpy as np
import matplotlib.pyplot as plt

def one_dimensional_random_walk(num_steps, p=0.5, step_length=1.0):
    position = 0.0
    trajectory = np.zeros(num_steps + 1)
    trajectory[0] = position
    
    for i in range(1, num_steps + 1):
        if np.random.random() < p:
            position += step_length
        else:
            position -= step_length
        
        trajectory[i] = position
    
    return position, trajectory

def verify_theoretical_relations(num_walks=1000, num_steps=1000, p_values=[0.5, 0.7]):
    results = {}
    step_length = 1.0
    
    for p in p_values:
        q = 1.0 - p
        
        theoretical_mean = (p - q) * step_length * num_steps
        theoretical_variance = 4 * step_length**2 * p * q * num_steps
        
        final_positions = np.zeros(num_walks)
        for i in range(num_walks):
            final_position, _ = one_dimensional_random_walk(num_steps, p, step_length)
            final_positions[i] = final_position
        
        experimental_mean = np.mean(final_positions)
        experimental_variance = np.var(final_positions)
        
        results[p] = {
            'theoretical_mean': theoretical_mean,
            'experimental_mean': experimental_mean,
            'mean_error': abs(theoretical_mean - experimental_mean) / abs(theoretical_mean) * 100 if theoretical_mean != 0 else abs(experimental_mean),
            'theoretical_variance': theoretical_variance,
            'experimental_variance': experimental_variance,
            'variance_error': abs(theoretical_variance - experimental_variance) / theoretical_variance * 100
        }
    
    return results

def plot_random_walk_examples(num_steps=1000, p_values=[0.5, 0.7]):
    plt.figure(figsize=(12, 6))
    
    for p in p_values:
        _, trajectory = one_dimensional_random_walk(num_steps, p)
        plt.plot(range(num_steps + 1), trajectory, label=f'p = {p}')
    
    plt.xlabel('Steps')
    plt.ylabel('Position')
    plt.title('Example Random Walk Trajectories')
    plt.legend()
    plt.grid(True)
    plt.savefig('random_walk_trajectories.png', dpi=300, bbox_inches='tight')
    plt.close()

def demonstrate_central_limit_theorem(num_walks=10000, num_steps=1000, p=0.5):
    final_positions = np.zeros(num_walks)
    
    for i in range(num_walks):
        final_position, _ = one_dimensional_random_walk(num_steps, p)
        final_positions[i] = final_position
    
    plt.figure(figsize=(10, 6))
    
    plt.hist(final_positions, bins=50, density=True, alpha=0.7, label='Simulation')
    
    x = np.linspace(min(final_positions), max(final_positions), 1000)
    theoretical_mean = (p - (1-p)) * num_steps
    theoretical_variance = 4 * p * (1-p) * num_steps
    gaussian = 1/np.sqrt(2*np.pi*theoretical_variance) * np.exp(-(x-theoretical_mean)**2/(2*theoretical_variance))
    plt.plot(x, gaussian, 'r-', label='Theoretical Gaussian')
    
    plt.xlabel('Final Position')
    plt.ylabel('Probability Density')
    plt.title('Distribution of Final Positions after Random Walks')
    plt.legend()
    plt.grid(True)
    plt.savefig('random_walk_distribution.png', dpi=300, bbox_inches='tight')
    plt.close()



results = verify_theoretical_relations()

print("Results for One-Dimensional Random Walk:")
for p, result in results.items():
    print(f"\nFor p = {p}:")
    print(f"Theoretical mean: {result['theoretical_mean']:.2f}")
    print(f"Experimental mean: {result['experimental_mean']:.2f}")
    print(f"Mean error: {result['mean_error']:.2f}%")
    print(f"Theoretical variance: {result['theoretical_variance']:.2f}")
    print(f"Experimental variance: {result['experimental_variance']:.2f}")
    print(f"Variance error: {result['variance_error']:.2f}%")

plot_random_walk_examples()

demonstrate_central_limit_theorem()

Results for One-Dimensional Random Walk:

For p = 0.5:
Theoretical mean: 0.00
Experimental mean: 1.82
Mean error: 1.82%
Theoretical variance: 1000.00
Experimental variance: 1040.47
Variance error: 4.05%

For p = 0.7:
Theoretical mean: 400.00
Experimental mean: 400.50
Mean error: 0.13%
Theoretical variance: 840.00
Experimental variance: 915.05
Variance error: 8.93%


In [69]:
def random_walk_with_traps(x0, trap_positions, max_steps=1000000):
    position = x0
    steps = 0
    
    while position not in trap_positions and steps < max_steps:
        if np.random.random() < 0.5:
            position += 1
        else:
            position -= 1
        
        steps += 1
    
    return steps, position

def calculate_average_lifetime(x0, trap_positions, num_walks=10000):
    lifetimes = np.zeros(num_walks)
    final_positions = np.zeros(num_walks)
    
    for i in range(num_walks):
        lifetime, final_position = random_walk_with_traps(x0, trap_positions)
        lifetimes[i] = lifetime
        final_positions[i] = final_position
    
    average_lifetime = np.mean(lifetimes)
    standard_error = np.std(lifetimes) / np.sqrt(num_walks)
    
    trap_stats = {}
    for trap in trap_positions:
        trap_stats[trap] = np.sum(final_positions == trap) / num_walks
    
    return average_lifetime, standard_error, trap_stats

def calculate_average_lifetime_enumeration(trap_positions, lattice_size=21):
    average_lifetimes = np.zeros(lattice_size)
    left_trap_probs = np.zeros(lattice_size)
    right_trap_probs = np.zeros(lattice_size)
    
    for trap in trap_positions:
        average_lifetimes[trap] = 0
        
        if trap == trap_positions[0]:
            left_trap_probs[trap] = 1.0
            right_trap_probs[trap] = 0.0
        else:
            left_trap_probs[trap] = 0.0
            right_trap_probs[trap] = 1.0
    
    non_trap_indices = [i for i in range(lattice_size) if i not in trap_positions]
    
    max_iterations = 10000
    tolerance = 1e-10
    
    for x in non_trap_indices:
        left_trap_probs[x] = 1.0 - (x - trap_positions[0]) / (trap_positions[1] - trap_positions[0])
        right_trap_probs[x] = 1.0 - left_trap_probs[x]
        average_lifetimes[x] = x * (lattice_size - 1 - x)
    
    for _ in range(max_iterations):
        max_change = 0
        for x in non_trap_indices:
            old_value = left_trap_probs[x]
            left_trap_probs[x] = 0.5 * (left_trap_probs[max(0, x-1)] + left_trap_probs[min(lattice_size-1, x+1)])
            max_change = max(max_change, abs(left_trap_probs[x] - old_value))
        
        if max_change < tolerance:
            break
    
    for x in non_trap_indices:
        right_trap_probs[x] = 1.0 - left_trap_probs[x]
    
    for _ in range(max_iterations):
        max_change = 0
        for x in non_trap_indices:
            old_value = average_lifetimes[x]
            average_lifetimes[x] = 1 + 0.5 * (average_lifetimes[max(0, x-1)] + average_lifetimes[min(lattice_size-1, x+1)])
            max_change = max(max_change, abs(average_lifetimes[x] - old_value))
        
        if max_change < tolerance:
            break
    
    return average_lifetimes, left_trap_probs, right_trap_probs

In [71]:
import time

def analytical_lifetime(x, L):
    return x * (L - x)

def analytical_left_prob(x, L):
    return (L - x) / L

def analytical_right_prob(x, L):
    return x / L

def generate_comparison_figure():
    lattice_size = 21
    center = lattice_size // 2
    trap_positions = [center - 10, center + 10]
    initial_positions = range(1, lattice_size - 1)

    mc_lifetimes = []
    mc_errors = []
    mc_left_probs = []
    mc_right_probs = []
    for x0 in initial_positions:
        avg_lt, err, stats = calculate_average_lifetime(x0, trap_positions, num_walks=10000)
        mc_lifetimes.append(avg_lt)
        mc_errors.append(err)
        mc_left_probs.append(stats[trap_positions[0]])
        mc_right_probs.append(stats[trap_positions[1]])

    enum_lifetimes, enum_left_probs, enum_right_probs = calculate_average_lifetime_enumeration(trap_positions, lattice_size)

    ana_lifetimes = [analytical_lifetime(x, lattice_size - 1) for x in initial_positions]
    ana_left_probs = [analytical_left_prob(x, lattice_size - 1) for x in initial_positions]
    ana_right_probs = [analytical_right_prob(x, lattice_size - 1) for x in initial_positions]

    mc_rel_error = [abs(mc - ana) / ana * 100 for mc, ana in zip(mc_lifetimes, ana_lifetimes)]
    enum_rel_error = [abs(enum - ana) / ana * 100 for enum, ana in zip(enum_lifetimes[1:-1], ana_lifetimes)]

    plt.figure(figsize=(10, 12))

    plt.subplot(3, 1, 1)
    plt.errorbar(initial_positions, mc_lifetimes, yerr=mc_errors, fmt='bo-', label='Monte Carlo')
    plt.plot(initial_positions, enum_lifetimes[1:-1], 'r^-', label='Enumeration')
    plt.plot(initial_positions, ana_lifetimes, 'g--', label='Analytical')
    plt.xlabel('Initial Position')
    plt.ylabel('Average Lifetime')
    plt.title('Average Lifetime vs Initial Position')
    plt.legend()
    plt.grid(True)

    plt.subplot(3, 1, 2)
    plt.plot(initial_positions, mc_left_probs, 'bo-', label='Monte Carlo (Left)')
    plt.plot(initial_positions, enum_left_probs[1:-1], 'b^-', label='Enumeration (Left)')
    plt.plot(initial_positions, ana_left_probs, 'b--', label='Analytical (Left)')
    plt.plot(initial_positions, mc_right_probs, 'ro-', label='Monte Carlo (Right)')
    plt.plot(initial_positions, enum_right_probs[1:-1], 'r^-', label='Enumeration (Right)')
    plt.plot(initial_positions, ana_right_probs, 'r--', label='Analytical (Right)')
    plt.xlabel('Initial Position')
    plt.ylabel('Absorption Probability')
    plt.title('Probability of Absorption at Left and Right Boundaries')
    plt.legend()
    plt.grid(True)

    plt.subplot(3, 1, 3)
    plt.plot(initial_positions, mc_rel_error, 'bo-', label='Monte Carlo')
    plt.plot(initial_positions, enum_rel_error, 'r^-', label='Enumeration')
    plt.xlabel('Initial Position')
    plt.ylabel('Relative Error (%)')
    plt.title('Relative Error Compared to Analytical Solution (Lifetime)')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.savefig('trap_problem_comparison.png', dpi=300, bbox_inches='tight')
    plt.close()

def generate_performance_figure():
    lattice_size = 21
    trap_positions = [0, 20]
    initial_position = 10
    num_walks_list = [100, 1000, 10000, 100000]

    mc_times = []
    for num_walks in num_walks_list:
        start_time = time.time()
        calculate_average_lifetime(initial_position, trap_positions, num_walks)
        mc_times.append(time.time() - start_time)

    start_time = time.time()
    calculate_average_lifetime_enumeration(trap_positions, lattice_size)
    enum_time = time.time() - start_time

    ana_time = 0.0001

    plt.figure(figsize=(8, 6))
    plt.semilogy(num_walks_list, mc_times, 'bo-', label='Monte Carlo')
    plt.axhline(y=enum_time, color='r', linestyle='--', label='Enumeration')
    plt.axhline(y=ana_time, color='g', linestyle='-.', label='Analytical')
    plt.xlabel('Number of Monte Carlo Simulations')
    plt.ylabel('Execution Time (s)')
    plt.title('Performance Comparison (Log Scale)')
    plt.legend()
    plt.grid(True)
    plt.savefig('trap_problem_performance.png', dpi=300, bbox_inches='tight')
    plt.close()


generate_comparison_figure()
generate_performance_figure()

In [ ]:
def random_walk_with_traps(x0, trap_positions, max_steps=1000000):
    position = x0
    steps = 0
    
    while position not in trap_positions and steps < max_steps:
        if np.random.random() < 0.5:
            position += 1
        else:
            position -= 1
        
        steps += 1
    
    return steps, position

def calculate_average_lifetime(x0, trap_positions, num_walks=10000):
    lifetimes = np.zeros(num_walks)
    final_positions = np.zeros(num_walks)
    
    for i in range(num_walks):
        lifetime, final_position = random_walk_with_traps(x0, trap_positions)
        lifetimes[i] = lifetime
        final_positions[i] = final_position
    
    average_lifetime = np.mean(lifetimes)
    standard_error = np.std(lifetimes) / np.sqrt(num_walks)
    
    trap_stats = {}
    for trap in trap_positions:
        trap_stats[trap] = np.sum(final_positions == trap) / num_walks
    
    return average_lifetime, standard_error, trap_stats

def analyze_trap_problem():
    lattice_size = 21
    center = lattice_size // 2
    trap_positions = [center - 10, center + 10]
    
    initial_positions = range(1, lattice_size - 1)
    lifetimes = []
    errors = []
    left_trap_probs = []
    right_trap_probs = []
    
    for x0 in initial_positions:
        avg_lifetime, error, trap_stats = calculate_average_lifetime(x0, trap_positions)
        lifetimes.append(avg_lifetime)
        errors.append(error)
        left_trap_probs.append(trap_stats.get(trap_positions[0], 0))
        right_trap_probs.append(trap_stats.get(trap_positions[1], 0))
        
        print(f"Initial position {x0}: Average lifetime = {avg_lifetime:.2f} ± {error:.2f}")
        for trap, prob in trap_stats.items():
            print(f"  Probability of being trapped at position {trap}: {prob:.4f}")
    
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 1, 1)
    plt.errorbar(initial_positions, lifetimes, yerr=errors, fmt='o-', capsize=3)
    plt.xlabel('Initial Position')
    plt.ylabel('Average Lifetime')
    plt.title('Average Lifetime of Random Walker Before Absorption')
    plt.grid(True)
    
    plt.subplot(2, 1, 2)
    plt.plot(initial_positions, left_trap_probs, 'bo-', label=f'Trap at {trap_positions[0]}')
    plt.plot(initial_positions, right_trap_probs, 'ro-', label=f'Trap at {trap_positions[1]}')
    plt.xlabel('Initial Position')
    plt.ylabel('Probability of Absorption')
    plt.title('Probability of Being Absorbed by Each Trap')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('trap_problem_analysis.png', dpi=300, bbox_inches='tight')
    plt.close()


analyze_trap_problem()

Initial position 1: Average lifetime = 19.19 ± 0.48
  Probability of being trapped at position 0: 0.9471
  Probability of being trapped at position 20: 0.0529
Initial position 2: Average lifetime = 36.46 ± 0.64
  Probability of being trapped at position 0: 0.9014
  Probability of being trapped at position 20: 0.0986
Initial position 3: Average lifetime = 50.61 ± 0.70
  Probability of being trapped at position 0: 0.8524
  Probability of being trapped at position 20: 0.1476
Initial position 4: Average lifetime = 63.89 ± 0.77
  Probability of being trapped at position 0: 0.7991
  Probability of being trapped at position 20: 0.2009
Initial position 5: Average lifetime = 75.72 ± 0.79
  Probability of being trapped at position 0: 0.7498
  Probability of being trapped at position 20: 0.2502
Initial position 6: Average lifetime = 82.76 ± 0.81
  Probability of being trapped at position 0: 0.7005
  Probability of being trapped at position 20: 0.2995
Initial position 7: Average lifetime = 91.35 ±

In [ ]:
def two_dimensional_random_walk(num_steps):
    x, y = 0, 0
    x_trajectory = np.zeros(num_steps + 1)
    y_trajectory = np.zeros(num_steps + 1)
    x_trajectory[0] = x
    y_trajectory[0] = y
    
    for i in range(1, num_steps + 1):
        direction = np.random.random()
        
        if direction < 0.25:
            x += 1
        elif direction < 0.5:
            x -= 1
        elif direction < 0.75:
            y += 1
        else:
            y -= 1
        
        x_trajectory[i] = x
        y_trajectory[i] = y
    
    return (x, y), x_trajectory, y_trajectory

def calculate_mean_squared_displacement(num_walks=1000, num_steps=1000, step_interval=10):
    time_points = np.arange(0, num_steps + 1, step_interval)
    num_measurements = len(time_points)
    
    squared_displacements = np.zeros((num_walks, num_measurements))
    
    for i in range(num_walks):
        _, x_traj, y_traj = two_dimensional_random_walk(num_steps)
        
        for j, t in enumerate(time_points):
            squared_displacements[i, j] = x_traj[t]**2 + y_traj[t]**2
    
    mean_squared_displacement = np.mean(squared_displacements, axis=0)
    std_error = np.std(squared_displacements, axis=0) / np.sqrt(num_walks)
    
    return time_points, mean_squared_displacement, std_error

def verify_scaling_law():
    time_points, msd, error = calculate_mean_squared_displacement()
    
    D = 0.25
    theoretical_msd = 4 * D * time_points
    
    relative_error = np.abs(msd - theoretical_msd) / theoretical_msd * 100
    
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 1, 1)
    plt.errorbar(time_points, msd, yerr=error, fmt='o', capsize=3, label='Simulation')
    plt.plot(time_points, theoretical_msd, 'r-', label='Theory: $\\langle r^2 \\rangle = 4Dt$')
    plt.xlabel('Time Steps')
    plt.ylabel('Mean Squared Displacement')
    plt.title('Mean Squared Displacement for 2D Random Walk')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(2, 1, 2)
    plt.plot(time_points[1:], relative_error[1:], 'bo-')
    plt.xlabel('Time Steps')
    plt.ylabel('Relative Error (%)')
    plt.title('Relative Error Between Simulation and Theory')
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('2d_random_walk_scaling.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    plt.figure(figsize=(10, 10))
    
    for i in range(5):
        _, x_traj, y_traj = two_dimensional_random_walk(1000)
        plt.plot(x_traj, y_traj, '-', alpha=0.7, label=f'Walk {i+1}')
    
    plt.xlabel('X Position')
    plt.ylabel('Y Position')
    plt.title('Sample Paths of 2D Random Walks')
    plt.grid(True)
    plt.axis('equal')
    plt.legend()
    
    plt.savefig('2d_random_walk_paths.png', dpi=300, bbox_inches='tight')
    plt.close()


verify_scaling_law()

/var/folders/km/y1gjflc56rd069hf7r__rftc0000gn/T/ipykernel_4028/2336296439.py:85: RuntimeWarning: invalid value encountered in divide
  relative_error = np.abs(msd - theoretical_msd) / theoretical_msd * 100


In [83]:
from matplotlib import colors

def initialize_linear_seed(grid_size, seed_length):
    grid = np.zeros(grid_size, dtype=int)
    
    start_pos = (grid_size[1] - seed_length) // 2
    grid[-1, start_pos:start_pos + seed_length] = 1
    
    return grid

def is_neighbor_to_cluster(grid, position):
    row, col = position
    height, width = grid.shape
    
    neighbors = [
        (row-1, col),
        (row+1, col),
        (row, col-1),
        (row, col+1)
    ]
    
    for r, c in neighbors:
        if 0 <= r < height and 0 <= c < width and grid[r, c] == 1:
            return True
    
    return False

def diffusion_limited_aggregation(grid_size=(200, 200), seed_length=100, num_particles=5000, max_distance=50):
    grid = initialize_linear_seed(grid_size, seed_length)
    
    time_grid = np.zeros_like(grid)
    
    cluster_height = grid_size[0] - 1
    
    for t in range(1, num_particles + 1):
        row = max(0, cluster_height - max_distance)
        col = np.random.randint(0, grid_size[1])
        
        while True:
            if is_neighbor_to_cluster(grid, (row, col)):
                grid[row, col] = 1
                time_grid[row, col] = t
                
                cluster_height = min(cluster_height, row)
                break
            
            direction = np.random.randint(0, 4)
            if direction == 0:
                row = max(0, row - 1)
            elif direction == 1:
                row = min(grid_size[0] - 1, row + 1)
            elif direction == 2:
                col = max(0, col - 1)
            else:
                col = min(grid_size[1] - 1, col + 1)
            
            if row - cluster_height > max_distance:
                row = max(0, cluster_height - np.random.randint(5, max_distance))
                col = np.random.randint(0, grid_size[1])
        
        if t % 100 == 0:
            print(f"Added {t} particles, cluster height: {cluster_height}")
    
    return grid, time_grid

def visualize_dla_cluster(grid, time_grid, save_path='dla_cluster.png'):
    plt.figure(figsize=(15, 10))
    
    plt.subplot(1, 2, 1)
    plt.imshow(grid, cmap='binary', origin='lower')
    plt.title('Final DLA Cluster')
    plt.colorbar(label='Cluster (1) / Empty (0)')
    
    plt.subplot(1, 2, 2)
    norm = colors.LogNorm(vmin=1, vmax=time_grid.max())
    plt.imshow(time_grid, cmap='viridis', norm=norm, origin='lower')
    plt.title('DLA Cluster Growth Timeline')
    plt.colorbar(label='Particle Addition Order')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()


print("Simulating Diffusion-Limited Aggregation...")
grid, time_grid = diffusion_limited_aggregation(seed_length=200)
visualize_dla_cluster(grid, time_grid)
print("DLA simulation completed.")

Simulating Diffusion-Limited Aggregation...
Added 100 particles, cluster height: 195
Added 200 particles, cluster height: 191
Added 300 particles, cluster height: 187
Added 400 particles, cluster height: 183
Added 500 particles, cluster height: 181
Added 600 particles, cluster height: 177
Added 700 particles, cluster height: 176
Added 800 particles, cluster height: 173
Added 900 particles, cluster height: 168
Added 1000 particles, cluster height: 165
Added 1100 particles, cluster height: 160
Added 1200 particles, cluster height: 157
Added 1300 particles, cluster height: 156
Added 1400 particles, cluster height: 151
Added 1500 particles, cluster height: 143
Added 1600 particles, cluster height: 139
Added 1700 particles, cluster height: 133
Added 1800 particles, cluster height: 130
Added 1900 particles, cluster height: 127
Added 2000 particles, cluster height: 123
Added 2100 particles, cluster height: 117
Added 2200 particles, cluster height: 113
Added 2300 particles, cluster height: 110

In [77]:
def initialize_central_seed(grid_size):
    grid = np.zeros(grid_size, dtype=int)
    
    center_row = grid_size[0] // 2
    center_col = grid_size[1] // 2
    grid[center_row, center_col] = 1
    
    return grid, (center_row, center_col)

def place_particle_on_boundary(center, radius, grid_size, boundary_shape='circle'):
    if boundary_shape == 'circle':
        angle = 2 * np.pi * np.random.random()
        
        row = int(center[0] + radius * np.sin(angle))
        col = int(center[1] + radius * np.cos(angle))
    
    elif boundary_shape == 'square':
        side = np.random.randint(0, 4)
        
        if side == 0:
            row = int(center[0] - radius)
            col = int(center[1] - radius + 2 * radius * np.random.random())
        elif side == 1:
            row = int(center[0] - radius + 2 * radius * np.random.random())
            col = int(center[1] + radius)
        elif side == 2:
            row = int(center[0] + radius)
            col = int(center[1] - radius + 2 * radius * np.random.random())
        else:
            row = int(center[0] - radius + 2 * radius * np.random.random())
            col = int(center[1] - radius)
    
    row = max(0, min(grid_size[0] - 1, row))
    col = max(0, min(grid_size[1] - 1, col))
    
    return (row, col)

def distance(pos1, pos2):
    return math.sqrt((pos1[0] - pos2[0])**2 + (pos1[1] - pos2[1])**2)

def dla_central_seed(grid_size=(400, 400), num_particles=5000, launch_radius_factor=1.5):
    grid, center = initialize_central_seed(grid_size)
    
    time_grid = np.zeros_like(grid)
    time_grid[center] = 1
    
    cluster_radius = 0
    
    for t in range(2, num_particles + 2):
        launch_radius = max(5, cluster_radius * launch_radius_factor)
        
        row, col = place_particle_on_boundary(center, launch_radius, grid_size, 'circle')
        
        max_distance = launch_radius * 2
        
        while True:
            if is_neighbor_to_cluster(grid, (row, col)):
                grid[row, col] = 1
                time_grid[row, col] = t
                
                new_radius = distance((row, col), center)
                cluster_radius = max(cluster_radius, new_radius)
                break
            
            direction = np.random.randint(0, 4)
            if direction == 0:
                row = max(0, row - 1)
            elif direction == 1:
                row = min(grid_size[0] - 1, row + 1)
            elif direction == 2:
                col = max(0, col - 1)
            else:
                col = min(grid_size[1] - 1, col + 1)
            
            if distance((row, col), center) > max_distance:
                row, col = place_particle_on_boundary(center, launch_radius, grid_size, 'circle')
        
        if t % 100 == 0:
            print(f"Added {t} particles, cluster radius: {cluster_radius:.2f}")
    
    return grid, time_grid, center

def dla_with_boundary_analysis(grid_size=(400, 400), num_particles=5000, 
                              launch_radius_factor=1.5, boundary_shape='circle'):
    grid, center = initialize_central_seed(grid_size)
    
    time_grid = np.zeros_like(grid)
    time_grid[center] = 1
    
    growth_snapshots = {}
    snapshot_points = [100, 500, 1000, 2000, 3000, 4000, 5000]
    
    cluster_radius = 0
    
    for t in range(2, num_particles + 2):
        launch_radius = max(5, cluster_radius * launch_radius_factor)
        
        row, col = place_particle_on_boundary(center, launch_radius, grid_size, boundary_shape)
        
        max_distance = launch_radius * 2
        
        steps_taken = 0
        max_steps = 10000
        
        while steps_taken < max_steps:
            if is_neighbor_to_cluster(grid, (row, col)):
                grid[row, col] = 1
                time_grid[row, col] = t
                
                new_radius = distance((row, col), center)
                cluster_radius = max(cluster_radius, new_radius)
                break
            
            direction = np.random.randint(0, 4)
            if direction == 0:
                row = max(0, row - 1)
            elif direction == 1:
                row = min(grid_size[0] - 1, row + 1)
            elif direction == 2:
                col = max(0, col - 1)
            else:
                col = min(grid_size[1] - 1, col + 1)
            
            if distance((row, col), center) > max_distance:
                row, col = place_particle_on_boundary(center, launch_radius, grid_size, boundary_shape)
            
            steps_taken += 1
        
        if t in snapshot_points:
            growth_snapshots[t] = grid.copy()
        
        if t % 100 == 0:
            print(f"Added {t} particles, cluster radius: {cluster_radius:.2f}")
    
    return grid, time_grid, center, growth_snapshots

def visualize_central_dla(grid, time_grid, center, save_path='dla_central_seed.png'):
    plt.figure(figsize=(15, 10))
    
    plt.subplot(1, 2, 1)
    plt.imshow(grid, cmap='binary')
    plt.scatter(center[1], center[0], c='red', s=50)
    plt.title('Final DLA Cluster with Central Seed')
    plt.colorbar(label='Cluster (1) / Empty (0)')
    
    plt.subplot(1, 2, 2)
    norm = colors.LogNorm(vmin=1, vmax=time_grid.max())
    plt.imshow(time_grid, cmap='viridis', norm=norm)
    plt.title('DLA Cluster Growth Timeline')
    plt.colorbar(label='Particle Addition Order')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    plt.figure(figsize=(12, 12))
    plt.imshow(grid, cmap='binary')
    plt.title('DLA Cluster with Central Seed')
    plt.axis('off')
    plt.savefig('dla_central_detailed.png', dpi=300, bbox_inches='tight')
    plt.close()

def analyze_cluster_properties(grid, center, boundary_shape="circle"):
    max_radius = min(grid.shape) // 2
    
    max_radius_for_analysis = 40
    radius_step = 2
    radius_values = np.arange(1, max_radius, radius_step)
    
    mass_values = []
    
    for r in radius_values:
        mass = 0
        for i in range(grid.shape[0]):
            for j in range(grid.shape[1]):
                if grid[i, j] == 1 and distance((i, j), center) <= r:
                    mass += 1
        mass_values.append(mass)
    
    analysis_mask = radius_values <= max_radius_for_analysis
    radius_for_fit = radius_values[analysis_mask]
    mass_for_fit = np.array(mass_values)[analysis_mask]
    
    log_radius = np.log(radius_for_fit)
    log_mass = np.log(mass_for_fit)
    
    slope, intercept, r_value, p_value, std_err = stats.linregress(log_radius, log_mass)
    
    plt.figure(figsize=(10, 6))
    
    plt.loglog(radius_values, mass_values, 'bo', alpha=0.5, label='All data')
    
    plt.loglog(radius_for_fit, mass_for_fit, 'ro', label='Data used for fitting')
    
    fit_x = np.linspace(min(log_radius), max(log_radius), 100)
    fit_y = intercept + slope * fit_x
    plt.loglog(np.exp(fit_x), np.exp(fit_y), 'g-', 
              label=f'Fit: dimension ≈ {slope:.3f} (R² = {r_value**2:.3f})')
    
    plt.xlabel('Radius')
    plt.ylabel('Mass (number of particles)')
    plt.title(f'DLA Fractal Dimension Analysis - {boundary_shape.capitalize()} Boundary')
    plt.legend()
    plt.grid(True)
    
    plt.annotate(f'Fitting limited to radius ≤ {max_radius_for_analysis} to avoid boundary effects', 
                xy=(0.5, 0.95), xycoords='figure fraction', 
                bbox=dict(boxstyle="round,pad=0.3", fc="yellow", alpha=0.3),
                ha='center', va='top')
    
    plt.savefig(f'dla_fractal_dimension_{boundary_shape}.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return {
        'fractal_dimension': slope,
        'r_squared': r_value**2,
        'radii': radius_values,
        'masses': mass_values,
        'fit_limit': max_radius_for_analysis
    }

def analyze_boundary_effects():
    print("\nAnalyzing boundary shape effects on DLA...")
    
    num_particles = 5000
    grid_size = (400, 400)
    
    print("Running DLA with circular boundary...")
    circle_grid, circle_time_grid, circle_center, circle_snapshots = dla_with_boundary_analysis(
        grid_size=grid_size, 
        num_particles=num_particles, 
        boundary_shape='circle'
    )
    
    print("Running DLA with square boundary...")
    square_grid, square_time_grid, square_center, square_snapshots = dla_with_boundary_analysis(
        grid_size=grid_size, 
        num_particles=num_particles, 
        boundary_shape='square'
    )
    
    plt.figure(figsize=(15, 10))
    
    plt.subplot(2, 2, 1)
    plt.imshow(circle_grid, cmap='binary')
    plt.title('DLA with Circular Boundary')
    plt.axis('off')
    
    plt.subplot(2, 2, 2)
    plt.imshow(square_grid, cmap='binary')
    plt.title('DLA with Square Boundary')
    plt.axis('off')
    
    plt.subplot(2, 2, 3)
    norm = colors.LogNorm(vmin=1, vmax=circle_time_grid.max())
    plt.imshow(circle_time_grid, cmap='viridis', norm=norm)
    plt.title('Growth Timeline (Circular Boundary)')
    plt.colorbar(label='Particle Addition Order')
    
    plt.subplot(2, 2, 4)
    norm = colors.LogNorm(vmin=1, vmax=square_time_grid.max())
    plt.imshow(square_time_grid, cmap='viridis', norm=norm)
    plt.title('Growth Timeline (Square Boundary)')
    plt.colorbar(label='Particle Addition Order')
    
    plt.tight_layout()
    plt.savefig('dla_boundary_comparison.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    selected_snapshots = [1000, 3000, 5000]
    
    plt.figure(figsize=(15, 10))
    for i, snapshot_point in enumerate(selected_snapshots):
        plt.subplot(2, 3, i+1)
        plt.imshow(circle_snapshots[snapshot_point], cmap='binary')
        plt.title(f'Circle Boundary: {snapshot_point} particles')
        plt.axis('off')
        
        plt.subplot(2, 3, i+4)
        plt.imshow(square_snapshots[snapshot_point], cmap='binary')
        plt.title(f'Square Boundary: {snapshot_point} particles')
        plt.axis('off')
    
    plt.tight_layout()
    plt.savefig('dla_growth_comparison.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    circle_dimensions = analyze_cluster_properties(circle_grid, circle_center, boundary_shape="circle")
    square_dimensions = analyze_cluster_properties(square_grid, square_center, boundary_shape="square")
    
    print(f"Estimated fractal dimension (circle): {circle_dimensions['fractal_dimension']:.3f}")
    print(f"Estimated fractal dimension (square): {square_dimensions['fractal_dimension']:.3f}")
    
    return {
        'circle': {
            'grid': circle_grid,
            'time_grid': circle_time_grid,
            'properties': circle_dimensions
        },
        'square': {
            'grid': square_grid,
            'time_grid': square_time_grid,
            'properties': square_dimensions
        }
    }

print("Simulating DLA with Central Seed...")

num_particles = 2000
grid_size = (300, 300)

grid, time_grid, center = dla_central_seed(grid_size=grid_size, num_particles=num_particles)
visualize_central_dla(grid, time_grid, center)

analyze_boundary_effects()

print(f"Completed. Added {num_particles} particles.")

Simulating DLA with Central Seed...
Added 100 particles, cluster radius: 15.30
Added 200 particles, cluster radius: 18.97
Added 300 particles, cluster radius: 27.00
Added 400 particles, cluster radius: 31.83
Added 500 particles, cluster radius: 36.07
Added 600 particles, cluster radius: 39.01
Added 700 particles, cluster radius: 42.45
Added 800 particles, cluster radius: 44.69
Added 900 particles, cluster radius: 48.10
Added 1000 particles, cluster radius: 50.33
Added 1100 particles, cluster radius: 52.40
Added 1200 particles, cluster radius: 54.56
Added 1300 particles, cluster radius: 55.66
Added 1400 particles, cluster radius: 59.14
Added 1500 particles, cluster radius: 61.13
Added 1600 particles, cluster radius: 65.00
Added 1700 particles, cluster radius: 65.80
Added 1800 particles, cluster radius: 68.01
Added 1900 particles, cluster radius: 70.80
Added 2000 particles, cluster radius: 74.89

Analyzing boundary shape effects on DLA...
Running DLA with circular boundary...
Added 100 p

In [ ]:
from collections import Counter
import time

def self_avoiding_random_walk(grid_size=100):
    grid = np.zeros((grid_size, grid_size), dtype=bool)
    
    x, y = grid_size // 2, grid_size // 2
    grid[x, y] = True
    
    x_traj = [x]
    y_traj = [y]
    
    directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    
    trapped = False
    while not trapped:
        available_moves = []
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < grid_size and 0 <= ny < grid_size and not grid[nx, ny]:
                available_moves.append((nx, ny))
        
        if not available_moves:
            trapped = True
            break
        
        next_pos = available_moves[np.random.randint(0, len(available_moves))]
        x, y = next_pos
        
        grid[x, y] = True
        x_traj.append(x)
        y_traj.append(y)
    
    return len(x_traj) - 1, x_traj, y_traj

def analyze_self_avoiding_walks(num_walks=10000, grid_size=100):
    print(f"Simulating {num_walks} self-avoiding walks...")
    start_time = time.time()
    
    lengths = []
    
    for i in range(num_walks):
        length, _, _ = self_avoiding_random_walk(grid_size)
        lengths.append(length)
        
        if (i + 1) % 1000 == 0:
            elapsed = time.time() - start_time
            remaining = (elapsed / (i + 1)) * (num_walks - i - 1)
            print(f"Completed {i+1}/{num_walks} walks. Time elapsed: {elapsed:.1f}s, Estimated remaining: {remaining:.1f}s")
    
    counter = Counter(lengths)
    max_length = max(lengths)
    distribution = [counter.get(i, 0) for i in range(max_length + 1)]
    
    probability_dist = [count / num_walks for count in distribution]
    
    print(f"Analysis completed in {time.time() - start_time:.1f} seconds.")
    
    return lengths, probability_dist

def visualize_saw_results(lengths, probability_dist):
    plt.figure(figsize=(15, 10))
    
    plt.subplot(2, 2, 1)
    plt.hist(lengths, bins=30, alpha=0.7, density=True)
    plt.xlabel('Walk Length')
    plt.ylabel('Probability Density')
    plt.title('Distribution of Self-Avoiding Walk Lengths')
    plt.grid(True)
    
    plt.subplot(2, 2, 2)
    plt.bar(range(len(probability_dist)), probability_dist, alpha=0.7)
    plt.xlabel('Walk Length')
    plt.ylabel('Probability')
    plt.title('Probability Distribution of Walk Lengths')
    plt.grid(True)
    
    plt.subplot(2, 2, 3)
    cumulative_dist = np.cumsum(probability_dist)
    plt.plot(range(len(cumulative_dist)), cumulative_dist, 'bo-')
    plt.xlabel('Walk Length')
    plt.ylabel('Cumulative Probability')
    plt.title('Cumulative Distribution of Walk Lengths')
    plt.grid(True)
    
    plt.subplot(2, 2, 4)
    
    non_zero_indices = [i for i, p in enumerate(probability_dist) if p > 0]
    x_values = [i for i in non_zero_indices]
    y_values = [probability_dist[i] for i in non_zero_indices]
    
    plt.loglog(x_values, y_values, 'bo', alpha=0.7)
    plt.xlabel('Walk Length (log)')
    plt.ylabel('Probability (log)')
    plt.title('Log-Log Plot of Walk Length Distribution')
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('saw_analysis.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    plt.figure(figsize=(15, 5))
    
    for i in range(3):
        plt.subplot(1, 3, i+1)
        length, x_traj, y_traj = self_avoiding_random_walk()
        plt.plot(x_traj, y_traj, 'b-', alpha=0.7)
        plt.plot(x_traj[0], y_traj[0], 'go', label='Start')
        plt.plot(x_traj[-1], y_traj[-1], 'ro', label='End')
        plt.title(f'Example SAW (Length = {length})')
        plt.grid(True)
        plt.legend()
    
    plt.tight_layout()
    plt.savefig('saw_examples.png', dpi=300, bbox_inches='tight')
    plt.close()


lengths, probability_dist = analyze_self_avoiding_walks(num_walks=5000)
visualize_saw_results(lengths, probability_dist)

Simulating 5000 self-avoiding walks...
Completed 1000/5000 walks. Time elapsed: 0.1s, Estimated remaining: 0.5s
Completed 2000/5000 walks. Time elapsed: 0.2s, Estimated remaining: 0.3s
Completed 3000/5000 walks. Time elapsed: 0.3s, Estimated remaining: 0.2s
Completed 4000/5000 walks. Time elapsed: 0.4s, Estimated remaining: 0.1s
Completed 5000/5000 walks. Time elapsed: 0.5s, Estimated remaining: 0.0s
Analysis completed in 0.5 seconds.


In [ ]:
def count_self_avoiding_walks(max_length):
    grid_size = 2 * max_length + 1
    
    center = grid_size // 2
    
    counts = [0] * (max_length + 1)
    counts[0] = 1
    
    visited = np.zeros((grid_size, grid_size), dtype=bool)
    visited[center, center] = True
    
    directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
    
    def dfs(x, y, length):
        if length == max_length:
            return
        
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            
            if 0 <= nx < grid_size and 0 <= ny < grid_size and not visited[nx, ny]:
                visited[nx, ny] = True
                
                counts[length + 1] += 1
                
                dfs(nx, ny, length + 1)
                
                visited[nx, ny] = False
    
    dfs(center, center, 0)
    
    return counts

def analyze_saw_growth(max_length=16):
    print(f"Counting self-avoiding walks up to length {max_length}...")
    start_time = time.time()
    
    saw_counts = count_self_avoiding_walks(max_length)
    
    elapsed = time.time() - start_time
    print(f"Enumeration completed in {elapsed:.2f} seconds.")
    
    rw_counts = [4**n if n > 0 else 1 for n in range(max_length + 1)]
    
    ratios = [saw / rw if rw > 0 else 0 for saw, rw in zip(saw_counts, rw_counts)]
    
    print("\nCounts of self-avoiding walks vs random walks:")
    print("Length | SAWs      | Random Walks | Ratio")
    print("-------|-----------|--------------|---------")
    for n in range(max_length + 1):
        print(f"{n:6d} | {saw_counts[n]:9d} | {rw_counts[n]:12d} | {ratios[n]:.8f}")
    
    plt.figure(figsize=(15, 10))
    
    plt.subplot(2, 2, 1)
    plt.semilogy(range(max_length + 1), saw_counts, 'bo-', label='Self-Avoiding Walks')
    plt.semilogy(range(max_length + 1), rw_counts, 'ro-', label='Random Walks')
    plt.xlabel('Walk Length')
    plt.ylabel('Number of Walks (log scale)')
    plt.title('Growth of Walk Counts')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(2, 2, 2)
    plt.plot(range(max_length + 1), ratios, 'go-')
    plt.xlabel('Walk Length')
    plt.ylabel('Ratio (SAW/RW)')
    plt.title('Ratio of Self-Avoiding to Random Walks')
    plt.grid(True)
    
    lengths = np.array(range(max_length//2, max_length + 1))
    log_saw_counts = np.log(saw_counts[max_length//2:])
    
    coeffs = np.polyfit(lengths, log_saw_counts, 1)
    growth_constant = np.exp(coeffs[0])
    
    plt.subplot(2, 2, 3)
    plt.plot(lengths, log_saw_counts, 'bo', label='Log(SAW counts)')
    plt.plot(lengths, coeffs[0] * lengths + coeffs[1], 'r-', 
             label=f'Fit: growth constant ≈ {growth_constant:.4f}')
    plt.xlabel('Walk Length')
    plt.ylabel('Log(Number of Walks)')
    plt.title('Estimating SAW Growth Constant')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('saw_enumeration.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return saw_counts, rw_counts, ratios, growth_constant


saw_counts, rw_counts, ratios, growth_constant = analyze_saw_growth(max_length=16)
print(f"Estimated SAW growth constant: {growth_constant:.6f}")

Counting self-avoiding walks up to length 16...
Enumeration completed in 9.08 seconds.

Counts of self-avoiding walks vs random walks:
Length | SAWs      | Random Walks | Ratio
-------|-----------|--------------|---------
     0 |         1 |            1 | 1.00000000
     1 |         4 |            4 | 1.00000000
     2 |        12 |           16 | 0.75000000
     3 |        36 |           64 | 0.56250000
     4 |       100 |          256 | 0.39062500
     5 |       284 |         1024 | 0.27734375
     6 |       780 |         4096 | 0.19042969
     7 |      2172 |        16384 | 0.13256836
     8 |      5916 |        65536 | 0.09027100
     9 |     16268 |       262144 | 0.06205750
    10 |     44100 |      1048576 | 0.04205704
    11 |    120292 |      4194304 | 0.02867985
    12 |    324932 |     16777216 | 0.01936746
    13 |    881500 |     67108864 | 0.01313537
    14 |   2374444 |    268435456 | 0.00884549
    15 |   6416596 |   1073741824 | 0.00597592
    16 |  17245332 |   429

In [ ]:
def persistent_random_walk(num_steps, persistence=0.7):
    x, y = 0, 0
    x_traj = [x]
    y_traj = [y]
    
    directions = [(1, 0), (0, 1), (-1, 0), (0, -1)]
    
    current_dir_idx = np.random.randint(0, 4)
    
    for _ in range(num_steps):
        if np.random.random() < persistence:
            pass
        else:
            current_dir_idx = np.random.randint(0, 4)
        
        dx, dy = directions[current_dir_idx]
        x += dx
        y += dy
        
        x_traj.append(x)
        y_traj.append(y)
    
    return x_traj, y_traj

def analyze_persistence_effect(num_walks=1000, num_steps=1000, persistence_values=[0.0, 0.3, 0.7, 0.9]):
    results = {}
    
    for p in persistence_values:
        print(f"Simulating walks with persistence = {p}...")
        
        final_x = np.zeros(num_walks)
        final_y = np.zeros(num_walks)
        squared_displacements = np.zeros((num_walks, num_steps + 1))
        
        for i in range(num_walks):
            x_traj, y_traj = persistent_random_walk(num_steps, p)
            
            final_x[i] = x_traj[-1]
            final_y[i] = y_traj[-1]
            
            for t in range(num_steps + 1):
                squared_displacements[i, t] = x_traj[t]**2 + y_traj[t]**2
        
        mean_squared_disp = np.mean(squared_displacements, axis=0)
        std_error = np.std(squared_displacements, axis=0) / np.sqrt(num_walks)
        
        results[p] = {
            'final_x': final_x,
            'final_y': final_y,
            'mean_squared_disp': mean_squared_disp,
            'std_error': std_error
        }
    
    return results

def visualize_persistence_results(results, num_steps):
    persistence_values = sorted(results.keys())
    
    plt.figure(figsize=(12, 8))
    time_points = np.arange(0, num_steps + 1)
    
    for p in persistence_values:
        msd = results[p]['mean_squared_disp']
        error = results[p]['std_error']
        plt.errorbar(time_points, msd, yerr=error, label=f'p = {p}', alpha=0.7, capsize=3)
    
    t_values = np.linspace(0, num_steps, 100)
    plt.plot(t_values, t_values, 'k--', label='Normal diffusion (~t)')
    
    plt.xlabel('Time Steps')
    plt.ylabel('Mean Squared Displacement')
    plt.title('Effect of Persistence on Mean Squared Displacement')
    plt.legend()
    plt.grid(True)
    plt.xscale('log')
    plt.yscale('log')
    
    plt.savefig('persistent_walk_msd.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    plt.figure(figsize=(15, 10))
    
    for i, p in enumerate(persistence_values):
        plt.subplot(2, 2, i+1)
        plt.scatter(results[p]['final_x'], results[p]['final_y'], alpha=0.3, s=5)
        plt.xlabel('X Position')
        plt.ylabel('Y Position')
        plt.title(f'Final Positions (p = {p})')
        plt.grid(True)
        plt.axis('equal')
    
    plt.tight_layout()
    plt.savefig('persistent_walk_positions.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    plt.figure(figsize=(15, 10))
    
    for i, p in enumerate(persistence_values):
        plt.subplot(2, 2, i+1)
        
        for _ in range(3):
            x_traj, y_traj = persistent_random_walk(num_steps, p)
            plt.plot(x_traj, y_traj, alpha=0.7)
        
        plt.xlabel('X Position')
        plt.ylabel('Y Position')
        plt.title(f'Sample Paths (p = {p})')
        plt.grid(True)
        plt.axis('equal')
    
    plt.tight_layout()
    plt.savefig('persistent_walk_paths.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    persistence_lengths = []
    for p in persistence_values:
        if p == 0:
            persistence_lengths.append(1)
        else:
            persistence_length = 1 / (1 - p)
            persistence_lengths.append(persistence_length)
    
    plt.figure(figsize=(10, 6))
    plt.plot(persistence_values, persistence_lengths, 'bo-')
    plt.xlabel('Persistence Parameter')
    plt.ylabel('Persistence Length')
    plt.title('Relationship Between Persistence Parameter and Persistence Length')
    plt.grid(True)
    
    plt.savefig('persistence_length.png', dpi=300, bbox_inches='tight')
    plt.close()


num_steps = 1000
persistence_values = [0.0, 0.3, 0.7, 0.9]

results = analyze_persistence_effect(num_walks=500, num_steps=num_steps, persistence_values=persistence_values)

visualize_persistence_results(results, num_steps)


Simulating walks with persistence = 0.0...
Simulating walks with persistence = 0.3...
Simulating walks with persistence = 0.7...
Simulating walks with persistence = 0.9...


In [82]:
def continuous_random_walk(num_steps, step_length=1.0, variable_length=False, step_distribution=None):
    x, y = 0, 0
    x_traj = [x]
    y_traj = [y]
    
    for _ in range(num_steps):
        theta = 2 * np.pi * np.random.random()
        
        if variable_length:
            if step_distribution == 'gaussian':
                r = np.random.normal(0, step_length)
            elif step_distribution == 'exponential':
                r = np.random.exponential(step_length)
            elif step_distribution == 'levy':
                r = np.random.pareto(1.5) * step_length
            else:
                r = 2 * step_length * np.random.random()
        else:
            r = step_length
        
        x += r * np.cos(theta)
        y += r * np.sin(theta)
        
        x_traj.append(x)
        y_traj.append(y)
    
    return x_traj, y_traj

def analyze_continuous_walks(num_walks=1000, num_steps=1000, models=[
    {'name': 'Fixed Length', 'variable': False},
    {'name': 'Gaussian', 'variable': True, 'distribution': 'gaussian'},
    {'name': 'Exponential', 'variable': True, 'distribution': 'exponential'},
    {'name': 'Levy', 'variable': True, 'distribution': 'levy'}
]):
    results = {}
    
    for model in models:
        print(f"Simulating {model['name']} random walk model...")
        
        final_x = np.zeros(num_walks)
        final_y = np.zeros(num_walks)
        squared_displacements = np.zeros((num_walks, num_steps + 1))
        
        for i in range(num_walks):
            x_traj, y_traj = continuous_random_walk(
                num_steps, 
                variable_length=model.get('variable', False),
                step_distribution=model.get('distribution', None)
            )
            
            final_x[i] = x_traj[-1]
            final_y[i] = y_traj[-1]
            
            for t in range(num_steps + 1):
                squared_displacements[i, t] = x_traj[t]**2 + y_traj[t]**2
        
        mean_squared_disp = np.mean(squared_displacements, axis=0)
        std_error = np.std(squared_displacements, axis=0) / np.sqrt(num_walks)
        
        results[model['name']] = {
            'final_x': final_x,
            'final_y': final_y,
            'mean_squared_disp': mean_squared_disp,
            'std_error': std_error
        }
    
    return results

def visualize_continuous_walk_results(results, num_steps):
    model_names = list(results.keys())
    
    plt.figure(figsize=(12, 8))
    time_points = np.arange(0, num_steps + 1)
    
    for model_name in model_names:
        msd = results[model_name]['mean_squared_disp']
        error = results[model_name]['std_error']
        plt.errorbar(time_points[::50], msd[::50], yerr=error[::50], 
                     label=model_name, alpha=0.7, capsize=3)
    
    t_values = np.linspace(0, num_steps, 100)
    plt.plot(t_values, t_values, 'k--', label='Normal diffusion (~t)')
    
    plt.xlabel('Time Steps')
    plt.ylabel('Mean Squared Displacement')
    plt.title('Mean Squared Displacement for Different Random Walk Models')
    plt.legend()
    plt.grid(True)
    plt.xscale('log')
    plt.yscale('log')
    
    plt.savefig('continuous_walk_msd.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    plt.figure(figsize=(15, 10))
    
    for i, model_name in enumerate(model_names):
        plt.subplot(2, 2, i+1)
        plt.scatter(results[model_name]['final_x'], results[model_name]['final_y'], 
                   alpha=0.3, s=5)
        plt.xlabel('X Position')
        plt.ylabel('Y Position')
        plt.title(f'Final Positions ({model_name})')
        plt.grid(True)
        plt.axis('equal')
    
    plt.tight_layout()
    plt.savefig('continuous_walk_positions.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    plt.figure(figsize=(15, 10))
    
    for i, model_name in enumerate(model_names):
        plt.subplot(2, 2, i+1)
        
        for _ in range(3):
            x_traj, y_traj = continuous_random_walk(
                num_steps,
                variable_length=('Fixed' not in model_name),
                step_distribution=model_name.lower() if 'Fixed' not in model_name else None
            )
            plt.plot(x_traj, y_traj, alpha=0.7)
        
        plt.xlabel('X Position')
        plt.ylabel('Y Position')
        plt.title(f'Sample Paths ({model_name})')
        plt.grid(True)
        plt.axis('equal')
    
    plt.tight_layout()
    plt.savefig('continuous_walk_paths.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    scaling_exponents = {}
    for model_name in model_names:
        msd = results[model_name]['mean_squared_disp']
        mid_point = len(time_points) // 2
        log_time = np.log(time_points[mid_point:])
        log_msd = np.log(msd[mid_point:])
        
        slope, _, _, _, _ = stats.linregress(log_time, log_msd)
        scaling_exponents[model_name] = slope
    
    plt.figure(figsize=(10, 6))
    plt.bar(model_names, [scaling_exponents[model] for model in model_names])
    plt.axhline(y=1.0, linestyle='--', color='r', label='Normal diffusion (α=1)')
    plt.xlabel('Random Walk Model')
    plt.ylabel('Scaling Exponent α')
    plt.title('Scaling Exponents for Different Random Walk Models')
    plt.legend()
    plt.grid(True, axis='y')
    
    plt.savefig('continuous_walk_scaling.png', dpi=300, bbox_inches='tight')
    plt.close()


num_steps = 10000

results = analyze_continuous_walks(num_walks=10000, num_steps=num_steps)

visualize_continuous_walk_results(results, num_steps)

Simulating Fixed Length random walk model...
Simulating Gaussian random walk model...
Simulating Exponential random walk model...
Simulating Levy random walk model...
